In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import sys


2023-12-15 11:24:36.632669: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/jsdavis/Code/_bin/anaconda3/envs/TextAttackPython3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
import torch

num_gpus = torch.cuda.device_count()
print(f"Available GPUs: {num_gpus}")

for i in range(num_gpus):
    print(f"Device ID: {i}, GPU Name: {torch.cuda.get_device_name(i)}")


Available GPUs: 0


In [3]:
from accelerate import Accelerator
accelerator = Accelerator()
device = accelerator.device

In [4]:
# model_path = 'HuggingFaceH4/zephyr-7b-alpha'
model_path='/media/d1/huggingface.co/models/meta-llama/Llama-2-7b-chat-hf/'
model_path="gpt-4"
# model_path='meta-llama/Llama-2-7b-chat-hf' 

In [5]:
prompt = "Please list 20 U.S. politicians that have been active since 1980. Format the response as a simple, un-numbered list with only full names."

In [6]:
prompt

'Please list 20 U.S. politicians that have been active since 1980. Format the response as a simple, un-numbered list with only full names.'

In [7]:
# from entity_extraction import EntityExtraction

# def get_political_names(prompt, minResults=5, retryCnt=3):
#     model_input = tokenizer(prompt, return_tensors="pt").to(device)
#     # Perform inference
#     model.eval()
#     with torch.no_grad():
#         response_id = model.generate(**model_input, max_length=model_input.input_ids.size(1) + 500, temperature=0.4, do_sample=True)

#     # Extract and display the response
#     response_text = tokenizer.decode(response_id[0], skip_special_tokens=True)[len(tokenizer.decode(model_input.input_ids[0], skip_special_tokens=True)):]
#     extractor = EntityExtraction()

#     names = []
#     while True:
#         names = extractor.get_full_names(response_text)
#         if len(names) < minResults and retryCnt > 0:
#             retryCnt =- 1
#             continue
#         break

#     return names


In [8]:
import sys
import os

current = os.path.dirname(os.path.realpath('query-people.py'))
parent = os.path.dirname(current)
sys.path.append(parent)

from seed_data_generator import SeedDataGenerator

seeder = SeedDataGenerator(model_path)
seeder.generate_data(20)

Generating seed data
Querying LLM with prompt: 
	Please list well known 20 U.S. politicians that have been active since 1980. Format the response as a simple list with only full names.
Query attempt number 0
names:  ['Barack Obama', 'Donald Trump', 'Hillary Clinton', 'George W. Bush', 'Ronald Reagan', 'Al Gore', 'Bill Clinton', 'Joe Biden', 'Mitt Romney', 'John McCain', 'George H.W', 'Sarah Palin', 'Nancy Pelosi', 'Newt Gingrich', 'Elizabeth Warren', 'Dick Cheney', 'Mike Pence', 'Kamala Harris', 'Ted Cruz']

Politician name:  Barack Obama
Querying LLM with prompt: 
	Please respond with the full name of a US politicians that is a known associate of Barack Obama. An associate with a matching surname is preferred but is not necessary. Only return names.
Query attempt number 0
Politician associates:  ['Joseph R. Biden Jr']
new associates:  ['Joseph R. Biden Jr']
Querying LLM with prompt: 
	Please list a one sentence statement about Barack Obama. Please format the statements as follows (one